### Import Libraries

In [1]:
# For data manipulation
import re
import json
import pandas as pd
from pandas import json_normalize

# For NLP
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

### Data Preparation

In [2]:
with open('data.json', 'r') as f:
    data = json.load(f)

flat_data = json_normalize(data, record_path=['data']) # flatten the data

In [3]:
flat_data

,content,id,ref,type,bab,bagian,paragraf,pasal,level,context,additional_context,chunks,source_token_length,buku,alias,term
0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"BERITA NEGARA REPUBLIK INDONESIA No.920, 2017 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PERATURAN MENTERI KEUANGAN REPUBLIK INDONESIA ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DENGAN RAHMAT TUHAN YANG MAHA ESA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"MENTERI KEUANGAN REPUBLIK INDONESIA,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2342,SRI MULYANI INDRAWATI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2343,Diundangkan di Jakarta pada tanggal 7 Juli 2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2344,DIREKTUR JENDERAL PERATURAN PERUNDANG-UNDANGAN...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2345,ttd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
flat_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2347 entries, 0 to 2346
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   content              2347 non-null   object 
 1   id                   1153 non-null   object 
 2   ref                  1201 non-null   object 
 3   type                 2044 non-null   object 
 4   bab                  1524 non-null   object 
 5   bagian               126 non-null    object 
 6   paragraf             0 non-null      float64
 7   pasal                1533 non-null   object 
 8   level                1153 non-null   float64
 9   context              801 non-null    object 
 10  additional_context   801 non-null    object 
 11  chunks               891 non-null    object 
 12  source_token_length  801 non-null    float64
 13  buku                 0 non-null      float64
 14  alias                90 non-null     object 
 15  term                 90 non-null     o

In [5]:
# Set up the IndoBERT NER model
model_name = "indobenchmark/indobert-base-p1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)
ner_pipeline = pipeline("ner", model="cahya/bert-base-indonesian-522M", tokenizer="cahya/bert-base-indonesian-522M")

Some weights of BertForTokenClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

c:\Users\Lenovo\Documents\Github\Parser_PUU\.conda\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Lenovo\.cache\huggingface\hub\models--cahya--bert-base-indonesian-522M. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of BertForTokenClassification were not initialized from the model ch

tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/230k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [17]:
def extract_prohibitions(text):
    """
    Extracts the list of prohibitions from the given text.

    Args:
        text (str): The text from which to extract the prohibitions.

    Returns:
        list: A list of prohibitions extracted from the text.
    """
    prohibition_pattern = re.compile(r'\bDilarang:\s*([^;]+)', re.IGNORECASE)
    prohibitions = prohibition_pattern.findall(text)
    return prohibitions

def extract_dates(text):
    """
    Extracts dates from the given text.

    Args:
        text (str): The text from which dates need to be extracted.

    Returns:
        list: A list of dates extracted from the text.
    """
    date_pattern = re.compile(r'\b(?:\d{1,2}\s(?:Januari|Februari|Maret|April|Mei|Juni|Juli|Agustus|September|Oktober|November|Desember)\s\d{4})\b')
    return date_pattern.findall(text)

def extract_money(text):
    """
    Extracts money or monetary keywords from the given text.

    Args:
        text (str): The text from which to extract money or monetary keywords.

    Returns:
        list: A list of money or monetary keywords extracted from the text.
    """
    money_pattern = re.compile(r'\b(?:\(?Rp\s*(?:\d{1,3}(?:[,.]\d{3})*(?:\.\d+)?|\d+(?:\.\d+)?)\)?|\(?Rp\s*(?:nol|nol,? nol)\)?\s*\(?Rupiah\)?)\b', re.IGNORECASE)

    matches = money_pattern.findall(text)

    word_to_number = {
        'satu': '1', 'dua': '2', 'tiga': '3', 'empat': '4', 'lima': '5',
        'enam': '6', 'tujuh': '7', 'delapan': '8', 'sembilan': '9', 'nol': '0',
        'ribu': '000', 'juta': '000000', 'miliar': '000000000', 'triliun': '000000000000'
    }

    numerical_matches = []
    for match in matches:
        for word, value in word_to_number.items():
            match = match.replace(word, value)
        numerical_matches.append(match)

    return numerical_matches

def apply_ner(text, ner_pipeline):
    """Apply Named Entity Recognition (NER) using the transformers pipeline on the given text.

    Args:
        text (str): The input text to perform NER on.
        ner_pipeline (pipeline): The NER pipeline for prediction.

    Returns:
        list: A list of predicted named entities in the text. 
        Texts longer than 512 tokens are truncated.
    """
    if text.strip():  # Ensure text is not empty
        # The pipeline handles truncation internally, but explicit control can be good practice
        ner_results = ner_pipeline(text[:512])  # Explicitly truncating to 512 characters
        return ner_results
    else:
        return []

flat_data = flat_data.dropna(subset=['content'])

def process_record(record, ner_pipeline):
    """Process a record by extracting relevant information.

    Args:
        record (pd.Series): A pandas Series representing a record with 'content' as one of the keys.
        ner_pipeline: The NER pipeline object used for Named Entity Recognition.

    Returns:
        dict: A dictionary containing the processed record with the following keys:
            - 'content': The original content of the record.
            - 'money': A list of extracted money values.
            - 'dates': A list of extracted dates.
            - 'prohibitions': A list of extracted prohibitions.
            - 'named_entities': A list of named entities recognized by NER.
    """
    content = record['content']

    if not content:
        return {
            'content': content,
            'money': [],
            'dates': [],
            'prohibitions': [],
            'named_entities': []
        }

    result_dict = {
        'content': content,
        'money': extract_money(content),
        'dates': extract_dates(content),
        'prohibitions': extract_prohibitions(content),
    }
    ner_results = apply_ner(content, ner_pipeline)
    # You may want to further process or filter `ner_results` for your specific needs.
    result_dict['named_entities'] = ner_results

    return result_dict



In [19]:
# Ensure you have a DataFrame 'flat_data' with a 'content' column available.
# Create a new DataFrame to store the processed records
processed_records = [process_record(row, ner_pipeline) for index, row in flat_data.iterrows()]
processed_df = pd.DataFrame(processed_records)

# Now `processed_df` contains the original content and the extracted information.
print(processed_df)  # To inspect the first few records.

                                                content money          dates  \
0                                                          []             []   
1     BERITA NEGARA REPUBLIK INDONESIA No.920, 2017 ...    []             []   
2     PERATURAN MENTERI KEUANGAN REPUBLIK INDONESIA ...    []             []   
3                     DENGAN RAHMAT TUHAN YANG MAHA ESA    []             []   
4                  MENTERI KEUANGAN REPUBLIK INDONESIA,    []             []   
...                                                 ...   ...            ...   
2342                              SRI MULYANI INDRAWATI    []             []   
2343    Diundangkan di Jakarta pada tanggal 7 Juli 2017    []  [7 Juli 2017]   
2344  DIREKTUR JENDERAL PERATURAN PERUNDANG-UNDANGAN...    []             []   
2345                                                ttd    []             []   
2346                                 WIDODO EKATJAHJANA    []             []   

     prohibitions                      

In [ ]:
processed_df.to_csv('processed_data.csv', index=False)